In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import random
from collections import Counter

def top_k_states_from_trans(trans, k=8):
    state_freq = Counter()
    for s, cnt in trans.items():
        state_freq[s] += sum(cnt.values())
    return [s for s, _ in state_freq.most_common(k)]

def random_states_from_trans(trans, k=8, seed=42):
    random.seed(seed)
    states = list(trans.keys())
    return random.sample(states, k)


In [5]:
def build_transition_matrix(trans, states):
    idx = {s: i for i, s in enumerate(states)}
    n = len(states)
    M = np.zeros((n, n), dtype=float)

    for s_from in states:
        row = trans.get(s_from, None)
        total = sum(row.values())

        i = idx[s_from]
        for s_to, c in row.items():
            if s_to in idx:
                j = idx[s_to]
                M[i, j] = c / total
    return M


In [6]:

def plot_transition_matrix(M, states, title=""):
    fig, ax = plt.subplots()
    fig.set_size_inches(8, 8)
    im = ax.imshow(M)
    fig.colorbar(im, ax=ax, rotation='vertical')

    ax.set_title(title)

    labels = [str(s) for s in states]
    ax.set_xticks(range(len(labels)))
    ax.set_yticks(range(len(labels)))
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)

    plt.tight_layout()
    plt.show()


In [ ]:
from pathlib import Path
import muspy


def self_similarity_midi_ticks(
    midi_path: str
):
    music = muspy.read_midi(midi_path)
    resolution = int(music.resolution)
    window_ticks = 4 * resolution

    notes = []
    tracks = music.tracks

    for tr in tracks:
        for n in tr.notes:
            notes.append((int(n.time), int(n.duration), int(n.pitch)))

    max_tick = max(t + d for t, d, _ in notes)
    n_windows = int(np.ceil(max_tick / window_ticks))
    chroma = np.zeros((n_windows, 12), dtype=float)

    for t, d, p in notes:
        start_w = t // window_ticks
        end_w = (t + d) // window_ticks
        pc = p % 12 # интересна нота по модулю октавы

        for w in range(start_w, min(end_w + 1, n_windows)):
            chroma[w, pc] += d

    norms = np.linalg.norm(chroma, axis=1, keepdims=True)
    chroma = chroma / np.maximum(norms, 1e-12)

    S = chroma @ chroma.T
    
    plt.figure()
    plt.imshow(S, origin="lower", aspect="auto")
    plt.colorbar()
    plt.title(f"Матрица самоподобия: {Path(midi_path).name}")
    plt.tight_layout()
    plt.show()

    return S


Матрица самоподобия написана чуть выше. Это матрица из статьи на хабре: https://habr.com/ru/companies/lanit/articles/455742/. MIDI-трек разбивается на последовательные временные окна (длиной, например, один такт, в нашем случае именно так). Для каждого окна считается chroma-вектор из 12 значений, показывающих, сколько времени в этом окне звучали ноты каждого класса высот (по модулю одной октавы). Полученные векторы нормализуются. Затем для каждой пары окон вычисляется косинусное сходство между их chroma-векторами. Так как вектора нормализованы, достаточно их перемножить.